In [58]:
import pandas as pd
from my_rymscraper.rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [59]:
mt = pd.read_csv("metacritic_data.csv")

In [60]:
# mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"];
lower_alnum = lambda s: "".join(x for x in s.lower() if x.isalnum())
mt["id"] = mt["Singer"].apply(lower_alnum) + mt["Release_Year"].apply(str) + mt["Album_Name"].apply(lambda s: lower_alnum(s)[:3] + lower_alnum(s)[-3:])
tags = zip(mt["id"], mt["Singer"], mt["Album_Name"])
# tags = ["Common - A Beautiful Revolution, Vol. 2"]
cols = ['Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages']
df = pd.DataFrame(columns=cols)

In [61]:
lower_alnum_dash = lambda s: "".join(x for x in s.replace(" ", "-").lower() if (x.isalnum() or x == "-" ))

max_iter = 1 # one iteration is enough for most cases, raise it if errors occur
iter = 0
while iter < max_iter:
  new_tags = []
  for uid, singer, album in tags:
    try:
      # Try url first
      url = "https://rateyourmusic.com/release/album/%s/%s/" % (lower_alnum_dash(singer), lower_alnum_dash(album))
      tag = singer + " - " + album
      album_infos = network.get_album_infos(url=url)
      if not album_infos:
        # Try search then
        album_infos = network.get_album_infos(name=tag) # get album by search
      
      if not album_infos:
        print("❌ ", tag)
        new_tags.append((tag,uid))
      else:
        print("✅ ", tag)
        album_infos["id"] = uid
        df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
    # except IndexError:
    except (IndexError, AttributeError):
      new_tags.append((tag,uid))
      print("❌ ", tag)

  tags = new_tags
  iter += 1

✅  Tangerine Dream - Raum


In [62]:
df

,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages,Type,Ranked,id
0,Raum,Tangerine Dream,25 February 2022,3.35\n \n/ 5.00.5\n\n ...,"Berlin School\nIDM, Downtempo, Ambient","atmospheric, space, instrumental, ethereal,...",NaN,"[In 256 Zeichen, You’re Always on Time, Along ...","[#84837f, #9a2e14, #ac9084, #5a2b11, #ea7317, ...",NaN,Album,#470 for 2022,tangerinedream2022rauaum


In [63]:
xdf = df.reset_index()
xdf = df[cols]
xdf.reset_index()
xdf.describe()

,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages
count,1,1,1,1,1,1,0,1,1,0
unique,1,1,1,1,1,1,0,1,1,0
top,Raum,Tangerine Dream,25 February 2022,3.35\n \n/ 5.00.5\n\n ...,"Berlin School\nIDM, Downtempo, Ambient","atmospheric, space, instrumental, ethereal,...",NaN,"[In 256 Zeichen, You’re Always on Time, Along ...","[#84837f, #9a2e14, #ac9084, #5a2b11, #ea7317, ...",NaN
freq,1,1,1,1,1,1,NaN,1,1,NaN


In [64]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/opt/homebrew/anaconda3/envs/edav/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
xdf = xdf[['Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme']]
xdf.describe()

,Language
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [66]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [67]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [68]:
xdf

,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Num_of_RYM_Ratings
0,Raum,Tangerine Dream,25 February 2022,3.35,"Berlin School\nIDM, Downtempo, Ambient","atmospheric, space, instrumental, ethereal,...",NaN,"[In 256 Zeichen, You’re Always on Time, Along ...","[#84837f, #9a2e14, #ac9084, #5a2b11, #ea7317, ...",415


In [69]:
xdf.to_csv("rym_add.csv")